In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from sklearn import preprocessing
from sklearn import model_selection
from keras.models import Sequential, Model
from keras.layers import Input, Conv2D, MaxPooling2D, Dense, Dropout, ZeroPadding2D, Flatten
from keras.optimizers import Adam
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
df_train = pd.read_json("../input/train.json")
df_test = pd.read_json("../input/test.json")
df_train.inc_angle = df_train.inc_angle.replace('na', 0)
df_train.inc_angle = df_train.inc_angle.astype(float).fillna(0.0)

In [ ]:
# Train data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in df_train["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in df_train["band_2"]])
x_train = np.concatenate([x_band1[:, :, :, np.newaxis], x_band2[:, :, :, np.newaxis]], axis=-1)
y_train = np.array(df_train["is_iceberg"])
print("xtrain:", x_train.shape)

# Test data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in df_test["band_1"]])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in df_test["band_2"]])
x_test = np.concatenate([x_band1[:, :, :, np.newaxis], x_band2[:, :, :, np.newaxis]], axis=-1)
print("xtest:", x_test.shape)

In [ ]:
X_train, X_valid, Y_train, Y_valid = model_selection.train_test_split(x_train, y_train, test_size=0.3, random_state=1)

In [ ]:
inp = Input(shape=(75, 75, 2))

conv1 = Conv2D(32, (3,3), padding='same', activation='relu', kernel_initializer='glorot_normal')(inp)
conv2 = Conv2D(32, (3,3), padding='same', activation='relu', kernel_initializer='glorot_normal')(conv1)
pool1 = MaxPooling2D((2,2), strides=(2,2))(conv2)

conv3 = Conv2D(32, (3,3), padding='same', activation='relu', kernel_initializer='glorot_normal')(pool1)
conv4 = Conv2D(32, (3,3), padding='same', activation='relu', kernel_initializer='glorot_normal')(conv3)
pool2 = MaxPooling2D((2,2), strides=(2,2))(conv4)

conv5 = Conv2D(32, (3,3), padding='same', activation='relu', kernel_initializer='glorot_normal')(pool2)
conv6 = Conv2D(32, (3,3), padding='same', activation='relu', kernel_initializer='glorot_normal')(conv5)
pool3 = MaxPooling2D((2,2), strides=(2,2))(conv6)

conv7 = Conv2D(64, (3,3), padding='same', activation='relu', kernel_initializer='glorot_normal')(pool3)
conv8 = Conv2D(64, (3,3), padding='same', activation='relu', kernel_initializer='glorot_normal')(conv7)
pool4 = MaxPooling2D((2,2), strides=(2,2))(conv8)

conv9 = Conv2D(64, (3,3), padding='same', activation='relu', kernel_initializer='glorot_normal')(pool4)
conv10 = Conv2D(64, (3,3), padding='same', activation='relu', kernel_initializer='glorot_normal')(conv9)
pool5 = MaxPooling2D((2,2), strides=(2,2))(conv10)

conv11 = Conv2D(128, (3,3), padding='same', activation='relu', kernel_initializer='glorot_normal')(pool5)
conv12 = Conv2D(128, (3,3), padding='same', activation='relu', kernel_initializer='glorot_normal')(conv11)
conv13 = Conv2D(128, (3,3), padding='same', activation='relu', kernel_initializer='glorot_normal')(conv12)
pool6 = MaxPooling2D((2,2), strides=(2,2))(conv13)

flat = Flatten()(pool6)

dense1 = Dense(1024, activation='relu', kernel_initializer='glorot_normal')(flat)
drop1 = Dropout(0.5)(dense1)
out = Dense(1, activation='sigmoid')(drop1)

In [ ]:
model = Model(inputs=inp, outputs=out)
optimizer = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy']) 
callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
model.summary()

In [ ]:
model.fit(X_train, Y_train, batch_size=32, epochs=40, validation_data=(X_valid, Y_valid), verbose=1)

In [ ]:
preds = model.predict(x_test, verbose=1)

In [ ]:
submission = pd.DataFrame({'id': df_test["id"], 'is_iceberg': preds.reshape((preds.shape[0]))})
submission.head(200)

In [ ]:
submission.to_csv("./submission.csv", index=False)